In [ ]:
from OCDB.data import DataLoader
from OCDB.model import ADM4, RPPN, SHP, TCDF, GVAR, NTiCD
import torch
import warnings
import numpy as np
import pandas as pd
from OCDB.metrics.Metrics import SHD_CPDAG, CSD, SID, CED
from castle.algorithms import TTPM, ICALiNGAM, DirectLiNGAM, GraNDAG, GOLEM, DAG_GNN, Notears, NotearsNonlinear

warnings.filterwarnings("ignore")


In [ ]:
# Static datasets
# Dataset: sachs, DWD, CCS_Data, Cad, Auto_mpg, Abalone, Ozone
columns, feature, ground_trues = DataLoader(name="CCS_Data").data()

def func(name):
    if name == "ICALiNGAM":
        model = ICALiNGAM(random_state=2, max_iter=1000, thresh=1)
    elif name == "DirectLiNGAM":
        model = DirectLiNGAM(thresh=0.0001, measure="kernel")
    elif name == "Notears":
        model = Notears(w_threshold=0.05)
    elif name == "NotearsNonlinear":
        model = NotearsNonlinear(max_iter=100, w_threshold=0.3, device_type="gpu", device_ids=0, rho_max=100)
    elif name == "DAG_GNN":
        model = DAG_GNN(encoder_hidden=256, decoder_hidden=256, lr=0.001, epochs=50, k_max_iter=25, 
                        encoder_dropout=0.5, decoder_dropout=0.0,
                        encoder_type="sem", decoder_type="mlp", device_type="gpu", device_ids=0)
    elif name == "GraNDAG":
        model = GraNDAG(input_dim=feature.shape[1], hidden_num=1, hidden_dim=10, batch_size=64)
    elif name == "GOLEM":
        model = GOLEM(lambda_1=0.02, lambda_2=5, graph_thres=0.3, device_type="gpu", device_ids=0, num_iter=20000)
    
    model.learn(feature)
    causal_matrix = model.causal_matrix
    _SHD = SHD_CPDAG(ground_trues, causal_matrix)
    _SED = CSD(ground_trues, causal_matrix)
    _SID = SID(ground_trues, causal_matrix)
    _CED = CED(ground_trues, causal_matrix, ground_trues.shape[0])
    print(f"{name}, SHD:{_SHD}, SED:{_SED}, SID:{_SID}, CED:{_CED}")

func("ICALiNGAM")
func("DirectLiNGAM")
func("Notears")
func("NotearsNonlinear")
func("DAG_GNN")
func("GraNDAG")
func("GOLEM")

In [ ]:
# Time-series Datasets
# Dataset: NetSim, fMRI, Finance]:
data = "Finance"
columns, feature, ground_trues = DataLoader(name=data).data()
if data == "fMRI":
    columns = columns[0]
    feature = feature[0]
    ground_trues = ground_trues[0]
elif data == "Finance":
    columns = columns[8]
    feature = feature[8]
    ground_trues = ground_trues[8]

model = ICALiNGAM(random_state=42, max_iter=1000, thresh=0.03)
model.learn(feature)
causal_matrix = model.causal_matrix
_SHD = SHD_CPDAG(ground_trues, causal_matrix)
_SED = CSD(ground_trues, causal_matrix)
_SID = SID(ground_trues, causal_matrix)
_CED = CED(ground_trues, causal_matrix, ground_trues.shape[0])
print(f"ICALiNGAM, SHD:{_SHD}, SED:{_SED}, SID:{_SID}, CED:{_CED}")

model = DirectLiNGAM(thresh=0.03)
model.learn(feature)
causal_matrix = model.causal_matrix
_SHD = model._eval(ground_trues, metric="SHD-C")
_SED = model._eval(ground_trues, metric="CSD")
_SID = model._eval(ground_trues, metric="SID")
_CED = model._eval(ground_trues, metric="CED")
print(f"DirectLiNGAM, SHD:{_SHD}, SED:{_SED}, SID:{_SID}, CED:{_CED}")

device = torch.device("cuda:0")
model = TCDF(device, len(columns), len(columns), kernel_size=128, hidden_layers=2, threshold=1)
model = model.to(device)
model.fit(feature, epochs=10, significance=0.5, lr=0.001)
_SHD = model._eval(ground_trues, metric="SHD-C")
_SED = model._eval(ground_trues, metric="CSD")
_SID = model._eval(ground_trues, metric="SID")
_CED = model._eval(ground_trues, metric="CED")
print(f"TCDF, SHD:{_SHD}, SED:{_SED}, SID:{_SID}, CED:{_CED}")

device = torch.device("cuda:0")
model = GVAR(device, len(columns), num_hidden_layers=1, hidden_layer_size=64, order=6)
model = model.to(device)
model.fit(feature, Q=20, alpha=0.5, epochs=10)
_SHD = model._eval(ground_trues, metric="SHD-C")
_SED = model._eval(ground_trues, metric="CSD")
_SID = model._eval(ground_trues, metric="SID")
_CED = model._eval(ground_trues, metric="CED")
print(f"GVAR, SHD:{_SHD}, SED:{_SED}, SID:{_SID}, CED:{_CED}")

device = torch.device("cuda:0")
model = NTiCD(device, len(columns), input_size=1, output_size=1, hidden_dim=64, n_layers=3)
model = model.to(device)
model.fit(feature, epochs=10, batch_size=64, lr=0.001, window_size=1)
_SHD = model._eval(ground_trues, metric="SHD-C")
_SED = model._eval(ground_trues, metric="CSD")
_SID = model._eval(ground_trues, metric="SID")
_CED = model._eval(ground_trues, metric="CED")
print(f"NTiCD, SHD:{_SHD}, SED:{_SED}, SID:{_SID}, CED:{_CED}")

In [ ]:
# Event Sequence
# Dataset: "Microwave24V", "Wireless", "Microwave25V"
data = "Microwave24V"

columns, event_table, ground_trues = DataLoader(name=data).data()
if data != "Wireless":
    new_event_table = event_table.loc[:, ["event_type", "time_stamp"]]
else:
    new_event_table = event_table
device = torch.device("cuda:0")

model = ADM4(device, len(columns), graph_threshold=0.05, max_iter=5, em_max_iter=5, rho=0.2, decay=1)
model = model.to(device)
model.fit(new_event_table, columns)
_SHD = model._eval(ground_trues, metric="SHD-C")
_SED = model._eval(ground_trues, metric="CSD")
_SID = model._eval(ground_trues, metric="SID")
_CED = model._eval(ground_trues, metric="CED")
print(f"ADM4, SHD:{_SHD}, SED:{_SED}, SID:{_SID}, CED:{_CED}")
    
model = RPPN(device, len(columns), init_scale=2, embedding_dim=64, hidden_size=64, graph_threshold=0.01, batch_size=64)
model = model.to(device)
model.fit(new_event_table, columns, epochs=20, num_workers=0, save_path="./", split_ratio=0.4   )
_SHD = model._eval(ground_trues, metric="SHD-C")
_SED = model._eval(ground_trues, metric="CSD")
_SID = model._eval(ground_trues, metric="SID")
_CED = model._eval(ground_trues, metric="CED")
print(f"RPPN, SHD:{_SHD}, SED:{_SED}, SID:{_SID}, CED:{_CED}")
    
model = SHP(device, len(columns), decay=3, time_interval=5, seed=42, reg=3, penalty="BIC")
model.fit(new_event_table, columns, threshold=0.9)
_SHD = model._eval(ground_trues, metric="SHD-C")
_SED = model._eval(ground_trues, metric="CSD")
_SID = model._eval(ground_trues, metric="SID")
_CED = model._eval(ground_trues, metric="CED")
print(f"SHP, SHD:{_SHD}, SED:{_SED}, SID:{_SID}, CED:{_CED}")

if data == "Wireless":
    node = [columns.index(event) for event in event_table["event_type"]]
    event = list(event_table["event_type"])
    timestamp = list(event_table["time_stamp"])
    event_dict = {"event": event,
                  "timestamp": timestamp,
                  "node": node}
    topology_matrix = np.mat(np.eye(len(node), dtype=int))
    max_hop = 0
else:
    node = list(event_table["node"])
    event = list(event_table["event_type"])
    timestamp = list(event_table["time_stamp"])
    event_dict = {"event": event,
                  "timestamp": timestamp,
                  "node": node}
    topology_matrix = np.load(f"./DCGL/data/dataset/{data}/Topology.npy")
    max_hop = 2
new_event_table = pd.DataFrame(event_dict)
model = TTPM(topology_matrix=topology_matrix, max_hop=max_hop)
model.learn(new_event_table)
causal_matrix = model.causal_matrix
_SHD = SHD_CPDAG(ground_trues, causal_matrix)
_SED = CSD(ground_trues, causal_matrix)
_SID = SID(ground_trues, causal_matrix)
_CED = CED(ground_trues, causal_matrix, ground_trues.shape[0])
print(f"THP, SHD:{_SHD}, SED:{_SED}, SID:{_SID}, CED:{_CED}")
